<a href="https://colab.research.google.com/github/misko/spf/blob/main/90_spf_figures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'prawnukkah'
!gcloud config set project {project_id}

Updated property [core/project].


In [3]:
! git clone https://github.com/misko/spf.git

Cloning into 'spf'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 208 (delta 113), reused 127 (delta 56), pack-reused 0
Receiving objects: 100% (208/208), 13.95 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [4]:
from smart_open import open
import pickle
session=pickle.load(
    open('gs://spf-data/sessions_sigma0.0_sources3_n8192_timesteps256_timeinterval0.3_dtbounce/session_00000001.pkl','rb'))
from spf.nn.rf_torch import labels_to_source_images,detector_positions_to_theta_grid,radio_to_image
#generate the images for the session
d={}
d['source_image_at_t']=labels_to_source_images(torch.from_numpy(session['source_positions_at_t'])[None],width)[0]
d['detector_theta_image_at_t']=detector_positions_to_theta_grid(session['detector_position_at_t'][None],width)[0]
d['radio_image_at_t']=radio_to_image(session['beam_former_outputs_at_t'][None],d['detector_theta_image_at_t'][None])[0]
d['radio_image_at_t_normed']=d['radio_image_at_t']/d['radio_image_at_t'].sum(axis=2,keepdims=True).sum(axis=3,keepdims=True)

NameError: ignored

In [1]:
session['width_at_t'].shape

NameError: ignored

In [635]:

import matplotlib.pyplot as plt 


def plot_trajectory(ax,
                    positions,
                    width,
                    ms=30,
                    steps_per_fade=10,
                    fadep=0.8,
                    c='b',
                    rings=False,
                    label=None):
  ax.set_xlim([0,width])
  ax.set_ylim([0,width])
  n_steps=positions.shape[0]//steps_per_fade
  if positions.shape[0]%steps_per_fade!=0:
    n_steps+=1
  alpha=1.0
  for n in np.arange(n_steps):
    start=positions.shape[0]-(n+1)*steps_per_fade
    end=start+steps_per_fade
    start=max(0,start)
    ax.plot( positions[start:end,0], positions[start:end,1],'--',alpha=alpha,color=c,label=label)
    alpha*=fadep
  
  ax.plot( positions[-1,0], positions[-1,1],'.',ms=ms,c=c)
  if rings:
    n=4
    for x in range(n):
      ax.plot( positions[-1,0], positions[-1,1],'.',ms=ms*(1.8**x),c=c,alpha=1/n)


In [636]:
!rm 01_basic*.png

width=session['width_at_t'][0][0]

filenames=[]
plt.ioff()
step_size=4
for idx in np.arange(1,256,step_size):
  fig=plt.figure(figsize=(12,12))
  axs=fig.subplots(2,2)
  for _a in [0,1]:
    for _b in [0,1]:
      if _a==0 and _b==1: 
        continue
      axs[_a,_b].set_xlabel("X (m)")
      axs[_a,_b].set_ylabel("Y (m)")

  axs[0,0].set_title("Position map")
  plot_trajectory(axs[0,0],session['detector_position_at_t'][:idx],width,ms=30,label='detector')
  for n in np.arange(session['source_positions_at_t'].shape[1]):
    rings=(session['broadcasting_positions_at_t'][idx,n,0]==1)
    plot_trajectory(axs[0,0],session['source_positions_at_t'][:idx,n],width,ms=15,c='r',rings=rings,label='emitter %d' % n)
  #axs[0,0].legend()
  handles, labels = axs[0,0].get_legend_handles_labels()
  by_label = dict(zip(labels, handles))
  axs[0,0].legend(by_label.values(), by_label.keys())
  #lets draw the radio
  axs[1,0].imshow(d['source_image_at_t'][idx,0].T,origin='lower')
  axs[1,0].set_title("Emitters as image at t=%d" % idx)

  axs[1,1].imshow(d['radio_image_at_t'][idx,0].T,origin='lower')
  axs[1,1].set_title("Radio feature at t=%d" % idx)

  axs[0,1].plot(session['thetas_at_t'][idx],session['beam_former_outputs_at_t'][idx])
  axs[0,1].set_title("Beamformer output at t=%d" % idx)
  fn='01_basic_frame_%04d.png' % idx
  filenames.append(fn)
  fig.savefig(fn)
  plt.close(fig)
plt.ion()

In [637]:

# importing libraries
import os
import cv2 
from PIL import Image

images=[]
for fn in filenames:
  images.append(Image.open(fn))

images[0].save('01_basic.gif',
               save_all = True, append_images = images[1:], 
               optimize = False, duration = 10)

In [638]:
!sudo apt-get install imagemagick

Reading package lists... Done
Building dependency tree       
Reading state information... Done
imagemagick is already the newest version (8:6.9.10.23+dfsg-2.1ubuntu11.7).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [640]:
!convert -resize 600x600 -delay 20 -loop 0 01_basic*.png 01_basic_take3.gif